In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('mhealth.db')

In [4]:
df = pd.read_csv('Mental_Health_Care_in_the_Last_4_Weeks.csv')
df.to_sql('mhealth.db', conn, if_exists='append', index=False)

10404